In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def calc_pcf_total(aorta_file, highest_n): 
    file = pd.read_excel(aorta_file, sheet_name=None, header = 0)
    pcf_bins = {i:0 for i in range(1,highest_n+1)} 
    triangles = file["Triangles"].iloc[:,:]
    vertex_connections = make_connection_dict(triangles) 
    norm_factor = calc_norm_factor_total(file) 
    pcs = file["Principal_Curvatures"]
    for step in range(1, highest_n+1): 
        verts = vertices_n_away(vertex_connections, step)
        for vertex in verts:
            tc1 = pcs["K1"][vertex] + pcs["K2"][vertex] 
            for other_vertex in verts[vertex]:
                tc2 = pcs["K1"][other_vertex] + pcs["K2"][other_vertex] 
                pcf_bins[step] += (tc1 * tc2)/len(verts[vertex])
    for step_num in pcf_bins:
        num_of_vertices = len(pcs["K1"])
        pcf_bins[step_num] /= (num_of_vertices*norm_factor) 
    x_vals, y_vals = [], []
    for x in pcf_bins:
        x_vals.append(x)
        y_vals.append(pcf_bins[x])
    return x_vals, y_vals
    
def calc_pcf_gaussian(aorta_file, highest_n): 
    file = pd.read_excel(aorta_file, sheet_name=None, header = 0)
    pcf_bins = {i:0 for i in range(1,highest_n+1)} 
    triangles = file["Triangles"].iloc[:,:]
    vertex_connections = make_connection_dict(triangles) 
    norm_factor = calc_norm_factor_gaussian(file) 
    pcs = file["Principal_Curvatures"]
    for step in range(1, highest_n+1): 
        verts = vertices_n_away(vertex_connections, step)
        for vertex in verts:
            gc1 = pcs["K1"][vertex] * pcs["K2"][vertex] 
            for other_vertex in verts[vertex]:
                gc2 = pcs["K1"][other_vertex] * pcs["K2"][other_vertex] 
                pcf_bins[step] += (gc1 * gc2)/len(verts[vertex])
    for step_num in pcf_bins:
        num_of_vertices = len(pcs["K1"])
        pcf_bins[step_num] /= (num_of_vertices*norm_factor) 
    x_vals, y_vals = [], []
    for x in pcf_bins:
        x_vals.append(x)
        y_vals.append(pcf_bins[x])
    return x_vals, y_vals

def make_connection_dict(triangles): 
    connections_dict = {}
    for index in range(len(triangles["T1"])):
        for triangle1 in {triangles["T1"][index], triangles["T2"][index], triangles["T3"][index]}:
            for triangle2 in {triangles["T1"][index], triangles["T2"][index], triangles["T3"][index]}:
                if triangle1 != triangle2:
                    if triangle1 in connections_dict:
                        connections_dict[triangle1].add(triangle2)
                    else:
                        connections_dict[triangle1] = {triangle2}
    return connections_dict

def vertices_n_away(connections_dict, n): 
    n_away_dict = {i: set() for i in connections_dict}
    for starting_vertex in connections_dict:
        visited = {starting_vertex}
        vertices_of_interest = {starting_vertex}
        new_verts_of_interest = set()
        for step in range(n):
            for vertex in vertices_of_interest:
                for next_vert in connections_dict[vertex]:
                    if (next_vert not in visited) and (next_vert not in vertices_of_interest):
                        new_verts_of_interest.add(next_vert)
                visited.add(vertex)
            vertices_of_interest = new_verts_of_interest
            new_verts_of_interest = set()
        for vertex in vertices_of_interest:
            n_away_dict[starting_vertex].add(vertex)
    return n_away_dict

def calc_norm_factor_gaussian(aorta_file):
    norm_factor = 0
    pcs = aorta_file["Principal_Curvatures"]
    num_of_vertices = len(pcs["K1"])
    for index in range(num_of_vertices):
        # multiply principal curvatures for GC
        norm_factor += (pcs["K1"][index] * pcs["K2"][index])**2
    return norm_factor/num_of_vertices

def calc_norm_factor_total(aorta_file):
    norm_factor = 0
    pcs = aorta_file["Principal_Curvatures"]
    num_of_vertices = len(pcs["K1"])
    for index in range(num_of_vertices):
        # add principal curvatures for TC
        norm_factor += (pcs["K1"][index] + pcs["K2"][index])**2
    return norm_factor/num_of_vertices

In [3]:
pcf_dict_gc= {}
pcf_dict_total= {}

In [4]:
for aorta_file in os.listdir("_____"): # specify directory
    if not aorta_file in pcf_dict_gc:
        pcf_gc = calc_pcf_gaussian("_____"+aorta_file, 40) # specify path to directory 
        pcf_dict_gc[aorta_file] = pcf_gc[0],pcf_gc[1]
        print(f"{aorta_file} - GC calculated")
    if not aorta_file in pcf_dict_total:
        pcf_tc = calc_pcf_total("_____"+aorta_file, 40) # specify path to directory 
        pcf_dict_total[aorta_file] = pcf_tc[0],pcf_tc[1]
        print(f"{aorta_file} - TC calculated")

# IMPORTANT:
# .to_excel() overwrites the whole workbook, save gc and tc calculations in separate files
        
pcf_gc_df = pd.DataFrame.from_dict(pcf_dict_gc)
pcf_gc_df.T.to_excel("_____",sheet_name="_____") # specify file and sheet name

pcf_tc_df = pd.DataFrame.from_dict(pcf_dict_tc)
pcf_tc_df.T.to_excel("_____",sheet_name="_____") # specify file and sheet name


FileNotFoundError: [Errno 2] No such file or directory: '_____'